In [1]:
import torch
print(torch.cuda.is_available())  # Kết quả: True (nếu GPU đã bật)
print(torch.cuda.get_device_name(0))  # Kiểm tra loại GPU

True
Tesla T4


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import torch

In [3]:
lora_config = LoraConfig(
    r=16,  # Rank của LoRA, tăng giá trị này giúp học nhiều hơn nhưng tốn VRAM
    lora_alpha=32,  # Hệ số điều chỉnh mức độ học của LoRA
    lora_dropout=0.05,  # Dropout giúp tránh overfitting
    bias="none",  # Không cập nhật bias để tiết kiệm tài nguyên
    task_type=TaskType.CAUSAL_LM,  # Loại tác vụ
)

In [15]:
model_path = "vilm/vinallama-7b"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

print("Mô hình đã tải thành công với LoRA!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Mô hình đã tải thành công với LoRA!


In [16]:
from peft import get_peft_model

for param in model.parameters():
    param.requires_grad = True

model = get_peft_model(model, lora_config)  # Kích hoạt LoRA
model.train()
model.print_trainable_parameters()


trainable params: 8,388,608 || all params: 6,863,974,400 || trainable%: 0.1222


In [17]:
import json
from datasets import Dataset

# Load dataset từ Kaggle input
data_path = "/kaggle/input/train-data/examples.json"

with open(data_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Kiểm tra dữ liệu đầu vào
assert isinstance(dataset, list), "Dataset JSON phải là danh sách các mục"

# Chuyển đổi dữ liệu sang dạng văn bản huấn luyện
train_texts = [
    f"Instruction:\n{item['instruction']}\n\nInput:\n{item['input']}\n\nOutput:\n{item['output']}"
    for item in dataset if "instruction" in item and "input" in item and "output" in item
]

# Tạo dataset từ Hugging Face Datasets
train_dataset = Dataset.from_dict({"text": train_texts})

# Cấu hình tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dữ liệu
def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],  
        padding="max_length",  # Có thể đổi thành "longest" nếu muốn linh hoạt hơn
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized["labels"] = tokenized["input_ids"].clone()  # ✅ Thêm labels
    return tokenized

# Tokenize dataset
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Kiểm tra kết quả
print(tokenized_dataset[0])

Map:   0%|          | 0/6599 [00:00<?, ? examples/s]

{'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

MICRO_BATCH_SIZE = 1  # ✅ Giảm xuống 1 để tiết kiệm VRAM
GRADIENT_ACCUMULATION_STEPS = 16  # ✅ Tổng batch size = 1 * 16 = 16
TRAIN_STEPS = 600  # ✅ Train lâu hơn
LEARNING_RATE = 1e-4  # ✅ Giảm để tránh overfitting

# Khởi tạo collator với tokenizer đã load
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, 
    padding=True
)

# Cấu hình TrainingArguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_ratio=0.03,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    logging_steps=50,
    optim='adamw_torch',
    save_strategy="steps",
    save_steps=500,  # ✅ Giảm số lần lưu checkpoint để tránh lãng phí bộ nhớ
    report_to="tensorboard",
    save_total_limit=1,
    fp16=True,  # ✅ Bật FP16 để tiết kiệm VRAM
    remove_unused_columns=False,  # ✅ Fix lỗi Trainer không nhận dữ liệu
    gradient_checkpointing=False,  # ✅ Giảm bộ nhớ tiêu thụ khi huấn luyện
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,  # ✅ Đảm bảo padding đúng
)

# Bắt đầu huấn luyện
trainer.train()

Step,Training Loss
50,2.311100
100,2.188500
150,2.131700
200,2.024300
250,1.977900


In [ ]:
def generate_query(question, history=[]):
    input_text = f"Instruction:\nTạo truy vấn dạng logic để lấy thông tin tương ứng với câu hỏi đã cho.\n\n Input:\nQuestion: {question}\n\n Output:\n"
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

    output_ids = model.generate(
        input_ids, 
        max_length=256,
        temperature=0.2,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id  # ✅ Dừng khi gặp eos_token
    )
    
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Cắt bỏ phần dư
    output_text = output_text.split("Output:")[-1].strip()
    output_text = output_text.split("Input:")[0].strip()  # ✅ Xóa phần dư thừa

    return output_text



In [ ]:
query1 = generate_query("Ai là người sáng lập ra Open AI?")
query2 = generate_query("Con của chồng Ranavalona I là ai?")
query3 = generate_query("Giáo viên hướng dẫn Shigeno Yasutsugu là ai?")

print("📌 Truy vấn 1:", query1)
print("📌 Truy vấn 2:", query2)
print("📌 Truy vấn 3:", query3)


In [ ]:
import os

output_dir = "/kaggle/working/model"
os.makedirs(output_dir, exist_ok=True)

# Lưu mô hình
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Mô hình đã được lưu tại: {output_dir}")

In [ ]:
!tar -czvf vinallama7b_model.tar.gz -C /kaggle/working model

In [ ]:
!!kaggle kernels output topunguyen/chatkbqa-vi -p /kaggle/working/



